In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125064")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125064
Azure region: southcentralus
Subscription id: 8bb47da5-84b5-43cf-bd4a-97928e5c9b08
Resource group: aml-quickstarts-125064


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name='cpucluster'

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    
"--C": choice(0.3 , 0.5, 1.0) , 
"--max_iter": choice(10, 50, 100, 120, 150) })

# Specify a Policy
policy =  BanditPolicy(slack_factor = 0.1, evaluation_interval=3, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory =
 '.', entry_script='train.py', 
   compute_target='cpucluster')


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig (estimator=est,
                             hyperparameter_sampling=ps,
                             policy= policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()

run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: 743c467e-fe3f-4d1e-af49-847a42a6ca86
Web View: https://ml.azure.com/experiments/udacity-project/runs/743c467e-fe3f-4d1e-af49-847a42a6ca86?wsid=/subscriptions/8bb47da5-84b5-43cf-bd4a-97928e5c9b08/resourcegroups/aml-quickstarts-125064/workspaces/quick-starts-ws-125064


ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

In [11]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()

print(best_run.get_details()['runDefinition']['arguments'])

print(best_run.get_file_names())

best_model= best_run.get_file_names()

os.makedirs('outputs', exist_ok=True)
joblib.dump(value=best_model, filename='outputs/hyperdrive_model.pkl')


['--C', '1', '--max_iter', '150']
['azureml-logs/55_azureml-execution-tvmps_d2d0480f4b271c0a0ae222afada89d22900572050137089ea98920d6b8664549_d.txt', 'azureml-logs/65_job_prep-tvmps_d2d0480f4b271c0a0ae222afada89d22900572050137089ea98920d6b8664549_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_d2d0480f4b271c0a0ae222afada89d22900572050137089ea98920d6b8664549_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/114_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_a157af88-6fe8-4f6a-a879-3a3ff741c34d.jsonl', 'logs/azureml/dataprep/python_span_l_a157af88-6fe8-4f6a-a879-3a3ff741c34d.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/chosen_model.pkl', 'outputs/diabetes_model.pkl', 'outputs/hyperdrive_model.pkl']


['outputs/hyperdrive_model.pkl']

from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###